# Skin Cancer Training using MONAI

## Overview

HAM10000("Human Against Machine with Machine with 10000 training images")은 [Harvard Dataverse](https://dataverse.harvard.edu/)에서  
호스팅하는 다양한 인구의 피부경 사진 이미지 데이터 세트입니다.  

다음과 같은 여러 진단 범주로 구성된 10015개의 이미지로 구성됩니다.  
- 광선 각화증 및 상피내 암종/보웬병(akiec) : Actinic keratoses and intraepithelial carcinoma / Bowen's disease
- 기저 세포 암종(bcc) : basal cell carcinoma
- 양성 각화증 유사 병변(태양 흑점/지루성 각화증 및 편평 태선 (bkl) : benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses)
- 피부섬유종(df) : dermatofibroma
- 흑색종(mel) : melanoma
- 멜라닌세포 모반(nv) : melanocytic nevi
- 혈관 병변(vasc) : vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage
 
(Actinic keratoses and intraepithelial carcinoma / Bowen's disease (akiec), basal cell carcinoma (bcc), benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, bkl), dermatofibroma (df), melanoma (mel), melanocytic nevi (nv) and vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, vasc).

이 예제에서는 Pytorch를 사용하여 [MONAI](http://monai.io) 프레임워크를 Amazon SageMaker에 통합하는 방법을 보여주고 불균형 데이터 세트 및 이미지 변환을 지원할 수 있는  
MONAI 사전 처리 변환의 예제 코드를 제공합니다.  
또한 이미지 분류를 위해 Densenet과 같은 MONAI 신경망 아키텍처를 호출하는 코드를 보여주고 SageMaker 내에서 모델을 훈련하고 제공하기 위한 Pytorch 코드의 구조를 탐색합니다.  
또한 HAM10000 데이터 세트를 사용하여 추론을 위한 모델 교육 및 호스팅 모두를 위한 컴퓨팅 인프라를 시작하고 관리하기 위한 SageMaker API 호출을 다룰 것입니다.  

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

이 노트북은 100GB의 EBS 및 conda_pytorch_p36 커널이 있는 ml.t2.medium 노트북 인스턴스에서 생성 및 테스트되었습니다.

아래 코드는 MONAI 프레임워크와 종속 패키지를 설치하고 환경 변수를 설정합니다.

In [ ]:
# Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0

!pip install -r source/requirements.txt

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
env_path = Path('.') / 'set.env'
load_dotenv(dotenv_path=env_path)

skin_cancer_bucket=os.environ.get('SKIN_CANCER_BUCKET')
skin_cancer_bucket_path=os.environ.get('SKIN_CANCER_BUCKET_PATH')
skin_cancer_files=os.environ.get('SKIN_CANCER_FILES')
skin_cancer_files_ext=os.environ.get('SKIN_CANCER_FILES_EXT')
base_dir = os.environ.get('BASE_DIR')

print('Skin Cancer Bucket: '+skin_cancer_bucket)
print('Skin Cancer Bucket Prefix: '+skin_cancer_bucket_path)
print('Skin Cancer Files: '+skin_cancer_files)
print('Skin Cancer Files Ext: '+skin_cancer_files_ext)
print('Base Dir: '+base_dir)

## HAM10000 Data Transformation

transform_data.ipynb는 dataverse_files.zip을 다운로드하고 메타데이터에서 트레이닝 및 검증 세트를 위한 클래스별로 디렉토리를 빌드하는 변환을 수행합니다.  
또한 데이터를 보강하여 트레이닝을 위해 클래스 전체에서 보다 균형 잡힌 데이터 세트를 생성합니다.  
스크립트는 변환된 데이터 세트 HAM10000.tar.gz를 모델 트레이닝을 위해 set.env에서 식별된 동일한 S3 버킷에 업로드합니다.

In [ ]:
%run source/transform_data.ipynb


## Data

### 변환된 HAM10000 데이터 세트에 대한 Sagemaker 세션 및 S3 위치 생성

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

inputs = sagemaker_session.upload_data(path=base_dir+'HAM10000.tar.gz', bucket=skin_cancer_bucket, key_prefix=skin_cancer_bucket_path)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## Train Model
### Training

```monai_skin_cancer.py``` 스크립트는 SageMaker 모델(모델을 로드하는 model_fn 함수)을 트레이닝하고 호스팅하는 데 필요한 모든 코드를 제공합니다.  
트레이닝 스크립트는 SageMaker 외부에서 실행할 수 있는 트레이닝 스크립트와 매우 유사하지만 다음과 같은 다양한 환경 변수를 통해 트레이닝 환경에 대한 유용한 속성에 액세스할 수 있습니다.

* SM_MODEL_DIR: 모델 아티팩트를 기록할 디렉터리의 경로를 나타내는 문자열입니다. 이러한 아티팩트는 모델 호스팅을 위해 S3에 업로드됩니다..
* SM_NUM_GPUS: 현재 컨테이너에서 사용 가능한 GPU 수입니다.
* SM_CURRENT_HOST: 컨테이너 네트워크에 있는 현재 컨테이너의 이름입니다.
* SM_HOSTS: 모든 호스트를 포함하는 JSON 인코딩 목록 .
* SM_CHANNEL_TRAINING: 'training' 채널의 데이터가 포함된 디렉토리 경로를 나타내는 문자열입니다.
  
학습 환경 변수에 대한 자세한 내용은 다음을 참조하십시오. 
[SageMaker Containers](https://github.com/aws/sagemaker-containers).

일반적인 트레이닝 스크립트는  
- 입력 채널에서 데이터를 로드하고, 
- 하이퍼파라미터로 트레이닝을 구성하고, 
- 모델을 트레이닝하고, 
- 나중에 호스팅할 수 있도록 모델을 model_dir에 저장합니다. 
- 하이퍼파라미터는 스크립트에 인수로 전달되며 argparse.ArgumentParser 인스턴스로 검색할 수 있습니다.

MONAI는 UNet, DenseNet, GAN 등과 같은 심층 신경망을 포함하며 대규모 의료 이미지 볼륨에 대한 슬라이딩 윈도우 추론을 제공합니다.  
피부암 이미지 분류 모델에서 우리는 손실을 측정하면서 30개의 epochs 동안 피부암 이미지에 대해 MONAI DenseNet 모델을 트레이닝 합니다.

In [ ]:
!pygmentize source/monai_skin_cancer.py

## Run training in SageMaker

`PyTorch` 클래스를 사용하면 SageMaker 인프라에서 트레이닝 function을 트레이닝 job으로 실행할 수 있습니다.  
트레이닝 스크립트, IAM 역할, 트레이닝 인스턴스 수, 트레이닝 인스턴스 유형 및 하이퍼파라미터를 사용하여 구성해야 합니다.  
이 경우 ```ml.p3.8xlarge``` 인스턴스에서 트레이닝 작업을 실행할 것입니다.  
그러나 이 예제는 하나 또는 여러 개의 CPU 또는 GPU 인스턴스(([사용가능한 인스턴스의 전체 목록](https://aws.amazon.com/sagemaker/pricing/instance-types/)))에서 실행할 수 있습니다.  
hyperparameters 매개변수는 학습 스크립트에 전달될 값의 사전입니다.  
위의 ```monai_skin_cancer.py```  스크립트에서 이러한 값에 액세스하는 방법을 볼 수 있습니다. 

In [6]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='monai_skin_cancer.py',
                    source_dir='source',
                    role=role,
                    framework_version='1.5.0',
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.p3.2xlarge',
                    hyperparameters={
                        'backend': 'gloo',
                        'epochs': 1
                    })

PyTorch 객체를 구성한 후 S3에 업로드한 HAM10000 데이터 세트를 사용하여 fit할 수 있습니다.  
SageMaker는 데이터를 로컬 파일 시스템에 다운로드하므로 트레이닝 스크립트는 디스크에서 데이터를 간단히 읽을 수 있습니다.

In [ ]:
estimator.fit({'train': inputs})

## HOST Model
### Create real-time endpoint

트레이닝 작업 이후, PyTorchPredictor를 구축하고 배포하기 위해 ``PyTorch`` estimator 객체를 사용합니다.  
그러면 추론을 수행하는 데 사용할 수 있는 Sagemaker 엔드포인트,즉 호스팅 된 예측 서비스가 생성됩니다.

위에서 언급했듯이 우리는 필요한 monai_skin_cancer.py 스크립트에 `model_fn`을 구현했습니다.  
[sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers)에 정의된  
`input_fn`, `predict_fn`, `output_fn` 및 `transform_fm`의 기본 구현을 사용할 것입니다.

배포 기능에 대한 arguments를 통해 엔드포인트에 사용할 인스턴스의 수와 유형을 설정할 수 있습니다.  
트레이닝 작업에 사용한 값과 같을 필요는 없습니다.  예를 들어 GPU 기반 인스턴스 세트에서 모델을 훈련한 다음 엔드포인트를 CPU 기반 인스턴스에 배포할 수 있지만  
모델을 다음과 유사한 CPU 모델로 반환하거나 저장해야 합니다. monai_skin_cancer.py에서는 ```ml.m5.xlarge``` 인스턴스에 모델을 배포합니다.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

### Load Validation Images for Inference 

In [ ]:
from PIL import Image

print('Load Test Images for Inference')
val_dir = os.path.join(base_dir, 'HAM10000/val_dir')
class_names = sorted([x for x in os.listdir(val_dir) if os.path.isdir(os.path.join(val_dir, x))])
num_class = len(class_names)
image_files = [[os.path.join(val_dir, class_name, x)
                for x in os.listdir(os.path.join(val_dir, class_name))[:1]] 
               for class_name in class_names]
image_file_list = []
image_label_list = []

for i, class_name in enumerate(class_names):
    image_file_list.extend(image_files[i])
    image_label_list.extend([i] * len(image_files[i]))
        
num_total = len(image_label_list)
image_width, image_height = Image.open(image_file_list[0]).size

### MONAI Transform Image using Compose and Skin Cancer Dataset

MONAI는 Dictionary 및 Array 형식을 모두 지원하는 변환을 가지고 있으며 의료 이미지의 고차원에 특화되어 있습니다.  
변환에는 자르기 및 패드, 강도, IO, 후처리, 공간 및 유틸리티(Crop & Pad, Intensity, IO, Post-processing, Spatial, and Utilities)와 같은 여러 범주가 포함됩니다.  
다음 발췌문에서 Compose 클래스는 일련의 이미지 변환을 함께 연결하고 이미지의 단일 텐서를 반환합니다.

In [10]:
import torch
from torch.utils.data import DataLoader
from source.skin_cancer_dataset import SkinCancerDataset
from monai.transforms import Compose, LoadPNG, Resize, AsChannelFirst, ScaleIntensity, ToTensor

val_transforms = Compose([
        LoadPNG(image_only=True),
        AsChannelFirst(channel_dim=2),
        ScaleIntensity(),
        Resize(spatial_size=(64,64)),
        ToTensor()
])
    
val_ds = SkinCancerDataset(image_file_list, image_label_list, val_transforms)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=1)

### Evaluate

이제 예측기를 사용하여 실시간 추론을 수행하여 피부암 이미지를 분류할 수 있습니다.

In [ ]:
print('Sample Inference Results By Class:')

for i, val_data in enumerate(val_loader):
    response = predictor.predict(val_data[0])
    actual_label = val_data[1]
    pred = torch.nn.functional.softmax(torch.tensor(response), dim=1)
    top_p, top_class = torch.topk(pred, 1)
    print('actual class: '+class_names[actual_label.numpy()[0]])
    print('predicted class: '+class_names[top_class])
    print('predicted class probablity: '+str(round(top_p.item(),2)))
    print()

### Remove endpoint (Optional)
예제를 완료한 후 모델을 호스팅하는 인스턴스를 해제하려면 예측 엔드포인트를 삭제하십시오.

In [ ]:
predictor.delete_endpoint()